# Tests

In [8]:
"""
Main BEHAVIOR demo replay entrypoint
"""

import argparse
import datetime
import os
import pprint

import bddl
import h5py
import numpy as np

import igibson
from igibson.activity.activity_base import iGBEHAVIORActivityInstance
from igibson.render.mesh_renderer.mesh_renderer_cpu import MeshRendererSettings
from igibson.render.mesh_renderer.mesh_renderer_vr import VrSettings
from igibson.simulator import Simulator
from igibson.utils.git_utils import project_git_info
from igibson.utils.ig_logging import IGLogReader, IGLogWriter
from igibson.utils.utils import parse_str_config


def print_class_info(obj, indent=0):
    # Get the class of the object
    cls = obj.__class__

    # Print the class name
    print(" " * indent + f"Class: {cls.__name__}")

    # Iterate through the object's attributes
    for attr_name, attr_value in vars(obj).items():
        print(" " * (indent + 2) + f"{attr_name}:")
        
        # If the attribute is an instance of a custom class, recurse
        if hasattr(attr_value, '__dict__'):
            print_class_info(attr_value, indent + 4)
        else:
            print(" " * (indent + 4) + f"{attr_value}")

    # Print information about methods
    methods = [method for method in dir(cls) if callable(getattr(cls, method)) and not method.startswith("__")]
    if methods:
        print(" " * (indent + 2) + "Methods:")
        for method in methods:
            print(" " * (indent + 4) + method)


def verify_determinism(in_log_path, out_log_path):
    is_deterministic = True
    with h5py.File(in_log_path) as original_file, h5py.File(out_log_path) as new_file:
        for obj in original_file["physics_data"]:
            for attribute in original_file["physics_data"][obj]:
                is_close = np.isclose(
                    original_file["physics_data"][obj][attribute], new_file["physics_data"][obj][attribute]
                )
                is_deterministic = is_deterministic and is_close.all()
                if not is_close.all():
                    print(
                        "Mismatch for obj {} with mismatched attribute {} starting at timestep {}".format(
                            obj, attribute, np.where(is_close == False)[0][0]
                        )
                    )
    return bool(is_deterministic)


def safe_replay_demo(*args, **kwargs):
    """Replays a demo, asserting that it was deterministic."""
    demo_statistics = replay_demo(*args, **kwargs)
    assert (
        demo_statistics["deterministic"] == True
    ), "Replay was not deterministic (or was executed with disable_save=True)."


# Define a class to mock argparse's Namespace object
class Args:
    def __init__(self, vr_log_path, vr_replay_log_path=None, frame_save_path=None, disable_save=False, profile=False, mode=None):
        self.vr_log_path = vr_log_path
        self.vr_replay_log_path = vr_replay_log_path
        self.frame_save_path = frame_save_path
        self.disable_save = disable_save
        self.profile = profile
        self.mode = mode

# Manually specify the arguments here
args = Args(
    vr_log_path=r"C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data\ig_dataset\tests\storing_food_0_Rs_int_2021-05-31_11-49-30.hdf5",
    frame_save_path=r"C:\Users\Bryan\Documents\Code\demo\saved_frames\storing_food",
    mode="simple"
)



def main():
    args = Args(vr_log_path=r"C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data\ig_dataset\tests\storing_food_0_Rs_int_2021-05-31_11-49-30.hdf5", frame_save_path=r"C:\Users\Bryan\Documents\Code\demo\saved_frames\storing_food", mode="simple"
)
    bddl.set_backend("iGibson")
    igbhvr_act_inst = replay_demo(
        args.vr_log_path,
        out_log_path=args.vr_replay_log_path,
        disable_save=args.disable_save,
        frame_save_path=args.frame_save_path,
        mode=args.mode,
        profile=args.profile,
    )
    
    return igbhvr_act_inst

In [67]:
def replay_demo(
    in_log_path,
    out_log_path=None,
    disable_save=False,
    frame_save_path=None,
    verbose=True,
    mode="headless",
    start_callbacks=[],
    step_callbacks=[],
    end_callbacks=[],
    profile=False,
    image_size=(1280, 720),
):
    """
    Replay a BEHAVIOR demo.

    Note that this returns, but does not check for determinism. Use safe_replay_demo to assert for determinism
    when using in scenarios where determinism is important.

    @param in_log_path: the path of the BEHAVIOR demo log to replay.
    @param out_log_path: the path of the new BEHAVIOR demo log to save from the replay.
    @param frame_save_path: the path to save frame images to. None to disable frame image saving.
    @param mode: which rendering mode ("headless", "simple", "vr"). In simple mode, the demo will be replayed with simple robot view.
    @param disable_save: Whether saving the replay as a BEHAVIOR demo log should be disabled.
    @param profile: Whether the replay should be profiled, with profiler output to stdout.
    @param start_callback: A callback function that will be called immediately before starting to replay steps. Should
        take a single argument, an iGBEHAVIORActivityInstance.
    @param step_callback: A callback function that will be called immediately following each replayed step. Should
        take a single argument, an iGBEHAVIORActivityInstance.
    @param end_callback: A callback function that will be called when replay has finished. Should take a single
        argument, an iGBEHAVIORActivityInstance.
    @param image_size: The image size that should be used by the renderer.
    @return if disable_save is True, returns None. Otherwise, returns a boolean indicating if replay was deterministic.
    """
    # HDR files for PBR rendering
    hdr_texture = os.path.join(igibson.ig_dataset_path, "scenes", "background", "probe_02.hdr")
    hdr_texture2 = os.path.join(igibson.ig_dataset_path, "scenes", "background", "probe_03.hdr")
    light_modulation_map_filename = os.path.join(
        igibson.ig_dataset_path, "scenes", "Rs_int", "layout", "floor_lighttype_0.png"
    )
    background_texture = os.path.join(igibson.ig_dataset_path, "scenes", "background", "urban_street_01.jpg")

    # VR rendering settings
    vr_rendering_settings = MeshRendererSettings(
        optimized=True,
        fullscreen=False,
        env_texture_filename=hdr_texture,
        env_texture_filename2=hdr_texture2,
        env_texture_filename3=background_texture,
        light_modulation_map_filename=light_modulation_map_filename,
        enable_shadow=True,
        enable_pbr=True,
        msaa=False,
        light_dimming_factor=1.0,
    )

    # Check mode
    assert mode in ["headless", "vr", "simple", "pbgui"]

    # Initialize settings to save action replay frames
    vr_settings = VrSettings(config_str=IGLogReader.read_metadata_attr(in_log_path, "/metadata/vr_settings"))
    vr_settings.set_frame_save_path(frame_save_path)

    task = IGLogReader.read_metadata_attr(in_log_path, "/metadata/atus_activity")
    if task is None:
        task = IGLogReader.read_metadata_attr(in_log_path, "/metadata/task_name")
    task_id = IGLogReader.read_metadata_attr(in_log_path, "/metadata/activity_definition")
    if task_id is None:
        task_id = IGLogReader.read_metadata_attr(in_log_path, "/metadata/task_instance")
    scene = IGLogReader.read_metadata_attr(in_log_path, "/metadata/scene_id")
    physics_timestep = IGLogReader.read_metadata_attr(in_log_path, "/metadata/physics_timestep")
    render_timestep = IGLogReader.read_metadata_attr(in_log_path, "/metadata/render_timestep")
    filter_objects = IGLogReader.read_metadata_attr(in_log_path, "/metadata/filter_objects")
    instance_id = IGLogReader.read_metadata_attr(in_log_path, "/metadata/instance_id")
    urdf_file = IGLogReader.read_metadata_attr(in_log_path, "/metadata/urdf_file")
    
    

    if urdf_file is None:
        urdf_file = "{}_task_{}_{}_0_fixed_furniture".format(scene, task, task_id)

    if instance_id is None:
        instance_id = 0

    logged_git_info = IGLogReader.read_metadata_attr(in_log_path, "/metadata/git_info")
    logged_git_info = parse_str_config(logged_git_info)
    git_info = project_git_info()
    pp = pprint.PrettyPrinter(indent=4)

    for key in logged_git_info.keys():
        if key not in git_info:
            print(
                "Warning: {} not present in current git info. It might be installed through PyPI, "
                "so its version cannot be validated.".format(key)
            )
            continue

        logged_git_info[key].pop("directory", None)
        git_info[key].pop("directory", None)
        if logged_git_info[key] != git_info[key] and verbose:
            print("Warning, difference in git commits for repo: {}. This may impact deterministic replay".format(key))
            print("Logged git info:\n")
            pp.pprint(logged_git_info[key])
            print("Current git info:\n")
            pp.pprint(git_info[key])

    # VR system settings
    s = Simulator(
        mode=mode,
        physics_timestep=physics_timestep,
        render_timestep=render_timestep,
        rendering_settings=vr_rendering_settings,
        vr_settings=vr_settings,
        image_width=image_size[0],
        image_height=image_size[1],
    )

    igbhvr_act_inst = iGBEHAVIORActivityInstance(task, task_id)
    igbhvr_act_inst.initialize_simulator(
        simulator=s,
        scene_id=scene,
        scene_kwargs={
            "urdf_file": urdf_file,
        },
        load_clutter=True,
        online_sampling=False,
    )
    vr_agent = igbhvr_act_inst.simulator.robots[0]
    log_reader = IGLogReader(in_log_path, log_status=False)

    log_writer = None
    if not disable_save:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        if out_log_path == None:
            out_log_path = "{}_{}_{}_{}_{}_replay.hdf5".format(task, task_id, scene, instance_id, timestamp)

        log_writer = IGLogWriter(
            s,
            log_filepath=out_log_path,
            task=igbhvr_act_inst,
            store_vr=False,
            vr_robot=vr_agent,
            profiling_mode=profile,
            filter_objects=filter_objects,
        )
        log_writer.set_up_data_storage()

    
    try:
        for callback in start_callbacks:
            callback(igbhvr_act_inst, log_reader)
        
        count = 0

        task_done = False
        while log_reader.get_data_left_to_read():
            
            count += 1

            igbhvr_act_inst.simulator.step(print_stats=profile)
            task_done |= igbhvr_act_inst.check_success()[0]
            
            
            
            #-----------------------------------------------------------------------------------------------
            
            #-----------------------------------------------------------------------------------------------

            # Set camera each frame
            if mode == "vr":
                log_reader.set_replay_camera(s)

            for callback in step_callbacks:
                callback(igbhvr_act_inst, log_reader)

            # Get relevant VR action data and update VR agent
            vr_agent.apply_action(log_reader.get_agent_action("vr_robot"))

            if not disable_save:
                log_writer.process_frame()

        print("Total frames: ", count)
        print("Demo was succesfully completed: ", task_done)

        demo_statistics = {}
        for callback in end_callbacks:
            callback(igbhvr_act_inst, log_reader)
    finally:
        s.disconnect()
        if not disable_save:
            log_writer.end_log_session()

    is_deterministic = None
    if not disable_save:
        is_deterministic = verify_determinism(in_log_path, out_log_path)
        print("Demo was deterministic: ", is_deterministic)

    demo_statistics = {
        "deterministic": is_deterministic,
        "task": task,
        "task_id": int(task_id),
        "scene": scene,
        "task_done": task_done,
        "total_frame_num": log_reader.total_frame_num,
    }
    return demo_statistics

In [43]:
def list_class_members(cls):
    attributes = []
    methods = []
    
    for name, member in cls.__dict__.items():
        if not name.startswith('__'):  # Exclude dunder methods
            if callable(member):
                methods.append(name)
            else:
                attributes.append(name)
    
    print(f"Attributes of {cls.__name__}:")
    for attr in attributes:
        print(f"- {attr}")
    
    print(f"\nMethods of {cls.__name__}:")
    for method in methods:
        print(f"- {method}")
        
        
import inspect

def inspect_class(cls, indent=0, max_depth=None):
    def print_indented(text):
        print("  " * indent + text)

    # Print the class name
    # print_indented(f"Class: {cls.__name__}")

    # Inspect attributes and methods
    for name, value in inspect.getmembers(cls):
        # Skip built-in attributes and methods
        if name.startswith("__") and name.endswith("__"):
            continue

        # Print the attribute or method name
        print_indented(f"- {name}: {type(value).__name__}")

        # If it's a method, print its signature
        if inspect.isfunction(value) or inspect.ismethod(value):
            signature = inspect.signature(value)
            print_indented(f"  Signature: {signature}")

        # If it's a nested class and we haven't reached max depth, recurse
        if inspect.isclass(value) and (max_depth is None or indent < max_depth):
            inspect_class(value, indent + 2, max_depth)

In [50]:
def replay_demo(
    in_log_path,
    out_log_path=None,
    disable_save=False,
    frame_save_path=None,
    verbose=True,
    mode="headless",
    start_callbacks=[],
    step_callbacks=[],
    end_callbacks=[],
    profile=False,
    image_size=(1280, 720),
):
    """
    Replay a BEHAVIOR demo.

    Note that this returns, but does not check for determinism. Use safe_replay_demo to assert for determinism
    when using in scenarios where determinism is important.

    @param in_log_path: the path of the BEHAVIOR demo log to replay.
    @param out_log_path: the path of the new BEHAVIOR demo log to save from the replay.
    @param frame_save_path: the path to save frame images to. None to disable frame image saving.
    @param mode: which rendering mode ("headless", "simple", "vr"). In simple mode, the demo will be replayed with simple robot view.
    @param disable_save: Whether saving the replay as a BEHAVIOR demo log should be disabled.
    @param profile: Whether the replay should be profiled, with profiler output to stdout.
    @param start_callback: A callback function that will be called immediately before starting to replay steps. Should
        take a single argument, an iGBEHAVIORActivityInstance.
    @param step_callback: A callback function that will be called immediately following each replayed step. Should
        take a single argument, an iGBEHAVIORActivityInstance.
    @param end_callback: A callback function that will be called when replay has finished. Should take a single
        argument, an iGBEHAVIORActivityInstance.
    @param image_size: The image size that should be used by the renderer.
    @return if disable_save is True, returns None. Otherwise, returns a boolean indicating if replay was deterministic.
    """
    # HDR files for PBR rendering
    hdr_texture = os.path.join(igibson.ig_dataset_path, "scenes", "background", "probe_02.hdr")
    hdr_texture2 = os.path.join(igibson.ig_dataset_path, "scenes", "background", "probe_03.hdr")
    light_modulation_map_filename = os.path.join(
        igibson.ig_dataset_path, "scenes", "Rs_int", "layout", "floor_lighttype_0.png"
    )
    background_texture = os.path.join(igibson.ig_dataset_path, "scenes", "background", "urban_street_01.jpg")

    # VR rendering settings
    vr_rendering_settings = MeshRendererSettings(
        optimized=True,
        fullscreen=False,
        env_texture_filename=hdr_texture,
        env_texture_filename2=hdr_texture2,
        env_texture_filename3=background_texture,
        light_modulation_map_filename=light_modulation_map_filename,
        enable_shadow=True,
        enable_pbr=True,
        msaa=False,
        light_dimming_factor=1.0,
    )

    # Check mode
    assert mode in ["headless", "vr", "simple", "pbgui"]

    # Initialize settings to save action replay frames
    vr_settings = VrSettings(config_str=IGLogReader.read_metadata_attr(in_log_path, "/metadata/vr_settings"))
    vr_settings.set_frame_save_path(frame_save_path)

    task = IGLogReader.read_metadata_attr(in_log_path, "/metadata/atus_activity")
    if task is None:
        task = IGLogReader.read_metadata_attr(in_log_path, "/metadata/task_name")
    task_id = IGLogReader.read_metadata_attr(in_log_path, "/metadata/activity_definition")
    if task_id is None:
        task_id = IGLogReader.read_metadata_attr(in_log_path, "/metadata/task_instance")
    scene = IGLogReader.read_metadata_attr(in_log_path, "/metadata/scene_id")
    physics_timestep = IGLogReader.read_metadata_attr(in_log_path, "/metadata/physics_timestep")
    render_timestep = IGLogReader.read_metadata_attr(in_log_path, "/metadata/render_timestep")
    filter_objects = IGLogReader.read_metadata_attr(in_log_path, "/metadata/filter_objects")
    instance_id = IGLogReader.read_metadata_attr(in_log_path, "/metadata/instance_id")
    urdf_file = IGLogReader.read_metadata_attr(in_log_path, "/metadata/urdf_file")
    
    

    if urdf_file is None:
        urdf_file = "{}_task_{}_{}_0_fixed_furniture".format(scene, task, task_id)

    if instance_id is None:
        instance_id = 0

    logged_git_info = IGLogReader.read_metadata_attr(in_log_path, "/metadata/git_info")
    logged_git_info = parse_str_config(logged_git_info)
    git_info = project_git_info()
    pp = pprint.PrettyPrinter(indent=4)

    for key in logged_git_info.keys():
        if key not in git_info:
            print(
                "Warning: {} not present in current git info. It might be installed through PyPI, "
                "so its version cannot be validated.".format(key)
            )
            continue

        logged_git_info[key].pop("directory", None)
        git_info[key].pop("directory", None)
        if logged_git_info[key] != git_info[key] and verbose:
            print("Warning, difference in git commits for repo: {}. This may impact deterministic replay".format(key))
            print("Logged git info:\n")
            pp.pprint(logged_git_info[key])
            print("Current git info:\n")
            pp.pprint(git_info[key])

    # VR system settings
    s = Simulator(
        mode=mode,
        physics_timestep=physics_timestep,
        render_timestep=render_timestep,
        rendering_settings=vr_rendering_settings,
        vr_settings=vr_settings,
        image_width=image_size[0],
        image_height=image_size[1],
    )

    igbhvr_act_inst = iGBEHAVIORActivityInstance(task, task_id)
    igbhvr_act_inst.initialize_simulator(
        simulator=s,
        scene_id=scene,
        scene_kwargs={
            "urdf_file": urdf_file,
        },
        load_clutter=True,
        online_sampling=False,
    )
    vr_agent = igbhvr_act_inst.simulator.robots[0]
    log_reader = IGLogReader(in_log_path, log_status=False)

    log_writer = None
    if not disable_save:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        if out_log_path == None:
            out_log_path = "{}_{}_{}_{}_{}_replay.hdf5".format(task, task_id, scene, instance_id, timestamp)

        log_writer = IGLogWriter(
            s,
            log_filepath=out_log_path,
            task=igbhvr_act_inst,
            store_vr=False,
            vr_robot=vr_agent,
            profiling_mode=profile,
            filter_objects=filter_objects,
        )
        log_writer.set_up_data_storage()

    

    for callback in start_callbacks:
        callback(igbhvr_act_inst, log_reader)

    task_done = False
    if log_reader.get_data_left_to_read():

        igbhvr_act_inst.simulator.step(print_stats=profile)
        task_done |= igbhvr_act_inst.check_success()[0]
        
        
        
        #-----------------------------------------------------------------------------------------------
        # for _, v in igbhvr_act_inst.object_scope.items():
        #     print(v)
        #     list_class_members(v)
        
        
        for obj_inst in igbhvr_act_inst.object_scope:
            print(obj_inst)
            print(type(obj_inst))
                

                
            #-----------------------------------------------------------------------------------------------

            # # Set camera each frame
            # if mode == "vr":
            #     log_reader.set_replay_camera(s)

            # for callback in step_callbacks:
            #     callback(igbhvr_act_inst, log_reader)

            # # Get relevant VR action data and update VR agent
            # vr_agent.apply_action(log_reader.get_agent_action("vr_robot"))

            # if not disable_save:
            #     log_writer.process_frame()

    #     print("Demo was succesfully completed: ", task_done)

    #     demo_statistics = {}
    #     for callback in end_callbacks:
    #         callback(igbhvr_act_inst, log_reader)
    # finally:
    #     s.disconnect()
    #     if not disable_save:
    #         log_writer.end_log_session()

    # is_deterministic = None
    # if not disable_save:
    #     is_deterministic = verify_determinism(in_log_path, out_log_path)
    #     print("Demo was deterministic: ", is_deterministic)

    # demo_statistics = {
    #     "deterministic": is_deterministic,
    #     "task": task,
    #     "task_id": int(task_id),
    #     "scene": scene,
    #     "task_done": task_done,
    #     "total_frame_num": log_reader.total_frame_num,
    # }
    
    return igbhvr_act_inst

In [68]:
igbhvr_act_inst = main()

Warning, difference in git commits for repo: iGibson. This may impact deterministic replay
Logged git info:

{   'branch_name': 'behavior',
    'code_diff': 'diff --git a/gibson2/render/glfw b/gibson2/render/glfw\n'
                 'index 96f9f5c4..40d150c3 160000\n'
                 '--- a/gibson2/render/glfw\n'
                 '+++ b/gibson2/render/glfw\n'
                 '@@ -1 +1 @@\n'
                 '-Subproject commit 96f9f5c4b970140ab5380fb9886ba787c8698937\n'
                 '+Subproject commit 40d150c39900921e94f16f53ba38b4b48689f80e\n'
                 'diff --git a/gibson2/render/pybind11 '
                 'b/gibson2/render/pybind11\n'
                 'index 38370a87..d159a563 160000\n'
                 '--- a/gibson2/render/pybind11\n'
                 '+++ b/gibson2/render/pybind11\n'
                 '@@ -1 +1 @@\n'
                 '-Subproject commit 38370a87f417001f3984cfa929860dabce6b69e7\n'
                 '+Subproject commit '
                 'd159a563383d

INFO:root:IndoorScene model: Rs_int
INFO:root:StaticIndoorScene scene: Rs_int
INFO:root:Category walls
INFO:root:Loading the following URDF template C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\urdf\Rs_int_walls.urdf
INFO:root:Scale: [1. 1. 1.]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scene_instances\20240902-175209_9347985240510938948_14300\walls_0.urdf
INFO:root:Category floors
INFO:root:Loading the following URDF template C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\urdf\Rs_int_floors.urdf
INFO:root:Scale: [1. 1. 1.]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scene_instances\20240902-175209_9347985240510938948_14300\floors_0.urdf
INFO:root:Category ceilings
INFO:root:Loading

SCENE NAME: Rs_int


INFO:root:Loading the following URDF template C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\objects\pot_plant\2ccd2b095bd8ec27e2ec5524ed185c34\2ccd2b095bd8ec27e2ec5524ed185c34.urdf
INFO:root:Scale: [0.56742843 0.61767094 0.64881942]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scene_instances\20240902-175209_9347985240510938948_14300\pot_plant_1_0.urdf
INFO:root:Category sofa
INFO:root:Loading the following URDF template C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\objects\sofa\427c7655012b6cc5593ebeeedbff73b\427c7655012b6cc5593ebeeedbff73b.urdf
INFO:root:Scale: [2.13580209 2.46916803 2.29157828]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scene_instances\20240902-175209_9347985240510938948_14300\sofa_2_0.urdf


Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\shape/visual\../../material/wall/COMBINED.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\shape/visual\../../material/wall/NORMAL.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\shape/visual\../../material/floor_2/COMBINED.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\shape/visual\../../material/floor_2/NORMAL.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\shape/visual\../../material/floor_1/COMBINED.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\shape/visual\../../material/floor_1/NORMAL.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\shape/visual\../../material/floor_0/COMBINED.png
Texture:  C:\Users\Bryan\Documen

In [65]:
def replay_demo(
    in_log_path,
    out_log_path=None,
    disable_save=False,
    frame_save_path=None,
    verbose=True,
    mode="headless",
    start_callbacks=[],
    step_callbacks=[],
    end_callbacks=[],
    profile=False,
    image_size=(1280, 720),
):
    """
    Replay a BEHAVIOR demo.

    Note that this returns, but does not check for determinism. Use safe_replay_demo to assert for determinism
    when using in scenarios where determinism is important.

    @param in_log_path: the path of the BEHAVIOR demo log to replay.
    @param out_log_path: the path of the new BEHAVIOR demo log to save from the replay.
    @param frame_save_path: the path to save frame images to. None to disable frame image saving.
    @param mode: which rendering mode ("headless", "simple", "vr"). In simple mode, the demo will be replayed with simple robot view.
    @param disable_save: Whether saving the replay as a BEHAVIOR demo log should be disabled.
    @param profile: Whether the replay should be profiled, with profiler output to stdout.
    @param start_callback: A callback function that will be called immediately before starting to replay steps. Should
        take a single argument, an iGBEHAVIORActivityInstance.
    @param step_callback: A callback function that will be called immediately following each replayed step. Should
        take a single argument, an iGBEHAVIORActivityInstance.
    @param end_callback: A callback function that will be called when replay has finished. Should take a single
        argument, an iGBEHAVIORActivityInstance.
    @param image_size: The image size that should be used by the renderer.
    @return if disable_save is True, returns None. Otherwise, returns a boolean indicating if replay was deterministic.
    """
    # HDR files for PBR rendering
    hdr_texture = os.path.join(igibson.ig_dataset_path, "scenes", "background", "probe_02.hdr")
    hdr_texture2 = os.path.join(igibson.ig_dataset_path, "scenes", "background", "probe_03.hdr")
    light_modulation_map_filename = os.path.join(
        igibson.ig_dataset_path, "scenes", "Rs_int", "layout", "floor_lighttype_0.png"
    )
    background_texture = os.path.join(igibson.ig_dataset_path, "scenes", "background", "urban_street_01.jpg")

    # VR rendering settings
    vr_rendering_settings = MeshRendererSettings(
        optimized=True,
        fullscreen=False,
        env_texture_filename=hdr_texture,
        env_texture_filename2=hdr_texture2,
        env_texture_filename3=background_texture,
        light_modulation_map_filename=light_modulation_map_filename,
        enable_shadow=True,
        enable_pbr=True,
        msaa=False,
        light_dimming_factor=1.0,
    )

    # Check mode
    assert mode in ["headless", "vr", "simple", "pbgui"]

    # Initialize settings to save action replay frames
    vr_settings = VrSettings(config_str=IGLogReader.read_metadata_attr(in_log_path, "/metadata/vr_settings"))
    vr_settings.set_frame_save_path(frame_save_path)

    task = IGLogReader.read_metadata_attr(in_log_path, "/metadata/atus_activity")
    if task is None:
        task = IGLogReader.read_metadata_attr(in_log_path, "/metadata/task_name")
    task_id = IGLogReader.read_metadata_attr(in_log_path, "/metadata/activity_definition")
    if task_id is None:
        task_id = IGLogReader.read_metadata_attr(in_log_path, "/metadata/task_instance")
    scene = IGLogReader.read_metadata_attr(in_log_path, "/metadata/scene_id")
    physics_timestep = IGLogReader.read_metadata_attr(in_log_path, "/metadata/physics_timestep")
    render_timestep = IGLogReader.read_metadata_attr(in_log_path, "/metadata/render_timestep")
    filter_objects = IGLogReader.read_metadata_attr(in_log_path, "/metadata/filter_objects")
    instance_id = IGLogReader.read_metadata_attr(in_log_path, "/metadata/instance_id")
    urdf_file = IGLogReader.read_metadata_attr(in_log_path, "/metadata/urdf_file")
    
    

    if urdf_file is None:
        urdf_file = "{}_task_{}_{}_0_fixed_furniture".format(scene, task, task_id)

    if instance_id is None:
        instance_id = 0

    logged_git_info = IGLogReader.read_metadata_attr(in_log_path, "/metadata/git_info")
    logged_git_info = parse_str_config(logged_git_info)
    git_info = project_git_info()
    pp = pprint.PrettyPrinter(indent=4)

    for key in logged_git_info.keys():
        if key not in git_info:
            print(
                "Warning: {} not present in current git info. It might be installed through PyPI, "
                "so its version cannot be validated.".format(key)
            )
            continue

        logged_git_info[key].pop("directory", None)
        git_info[key].pop("directory", None)
        if logged_git_info[key] != git_info[key] and verbose:
            print("Warning, difference in git commits for repo: {}. This may impact deterministic replay".format(key))
            print("Logged git info:\n")
            pp.pprint(logged_git_info[key])
            print("Current git info:\n")
            pp.pprint(git_info[key])

    # VR system settings
    s = Simulator(
        mode=mode,
        physics_timestep=physics_timestep,
        render_timestep=render_timestep,
        rendering_settings=vr_rendering_settings,
        vr_settings=vr_settings,
        image_width=image_size[0],
        image_height=image_size[1],
    )

    igbhvr_act_inst = iGBEHAVIORActivityInstance(task, task_id)
    igbhvr_act_inst.initialize_simulator(
        simulator=s,
        scene_id=scene,
        scene_kwargs={
            "urdf_file": urdf_file,
        },
        load_clutter=True,
        online_sampling=False,
    )
    vr_agent = igbhvr_act_inst.simulator.robots[0]
    log_reader = IGLogReader(in_log_path, log_status=False)

    log_writer = None
    if not disable_save:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        if out_log_path == None:
            out_log_path = "{}_{}_{}_{}_{}_replay.hdf5".format(task, task_id, scene, instance_id, timestamp)

        log_writer = IGLogWriter(
            s,
            log_filepath=out_log_path,
            task=igbhvr_act_inst,
            store_vr=False,
            vr_robot=vr_agent,
            profiling_mode=profile,
            filter_objects=filter_objects,
        )
        log_writer.set_up_data_storage()

    
    try:
        for callback in start_callbacks:
            callback(igbhvr_act_inst, log_reader)

        task_done = False
        while log_reader.get_data_left_to_read():

            igbhvr_act_inst.simulator.step(print_stats=profile)
            task_done |= igbhvr_act_inst.check_success()[0]
            
            
            
            #-----------------------------------------------------------------------------------------------
            storage = DynamicListStorage()
            storage.add_list(get_flattened_scene_graph(igbhvr_act_inst))
            print("called once")
            #-----------------------------------------------------------------------------------------------

            # Set camera each frame
            if mode == "vr":
                log_reader.set_replay_camera(s)

            for callback in step_callbacks:
                callback(igbhvr_act_inst, log_reader)

            # Get relevant VR action data and update VR agent
            vr_agent.apply_action(log_reader.get_agent_action("vr_robot"))

            if not disable_save:
                log_writer.process_frame()

        print("Demo was succesfully completed: ", task_done)

        demo_statistics = {}
        for callback in end_callbacks:
            callback(igbhvr_act_inst, log_reader)
    finally:
        s.disconnect()
        if not disable_save:
            log_writer.end_log_session()

    is_deterministic = None
    if not disable_save:
        is_deterministic = verify_determinism(in_log_path, out_log_path)
        print("Demo was deterministic: ", is_deterministic)

    demo_statistics = {
        "deterministic": is_deterministic,
        "task": task,
        "task_id": int(task_id),
        "scene": scene,
        "task_done": task_done,
        "total_frame_num": log_reader.total_frame_num,
    }
    return storage

In [66]:
storage = main()

Warning, difference in git commits for repo: iGibson. This may impact deterministic replay
Logged git info:

{   'branch_name': 'behavior',
    'code_diff': 'diff --git a/gibson2/render/glfw b/gibson2/render/glfw\n'
                 'index 96f9f5c4..40d150c3 160000\n'
                 '--- a/gibson2/render/glfw\n'
                 '+++ b/gibson2/render/glfw\n'
                 '@@ -1 +1 @@\n'
                 '-Subproject commit 96f9f5c4b970140ab5380fb9886ba787c8698937\n'
                 '+Subproject commit 40d150c39900921e94f16f53ba38b4b48689f80e\n'
                 'diff --git a/gibson2/render/pybind11 '
                 'b/gibson2/render/pybind11\n'
                 'index 38370a87..d159a563 160000\n'
                 '--- a/gibson2/render/pybind11\n'
                 '+++ b/gibson2/render/pybind11\n'
                 '@@ -1 +1 @@\n'
                 '-Subproject commit 38370a87f417001f3984cfa929860dabce6b69e7\n'
                 '+Subproject commit '
                 'd159a563383d

INFO:root:IndoorScene model: Rs_int
INFO:root:StaticIndoorScene scene: Rs_int
INFO:root:Category walls
INFO:root:Loading the following URDF template C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\urdf\Rs_int_walls.urdf
INFO:root:Scale: [1. 1. 1.]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scene_instances\20240828-135717_12121768275794561779_14300\walls_0.urdf
INFO:root:Category floors
INFO:root:Loading the following URDF template C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\urdf\Rs_int_floors.urdf
INFO:root:Scale: [1. 1. 1.]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scene_instances\20240828-135717_12121768275794561779_14300\floors_0.urdf
INFO:root:Category ceilings
INFO:root:Loadi

SCENE NAME: Rs_int


INFO:root:Instantiating scene into the following urdfs:
INFO:root:C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scene_instances\20240828-135717_12121768275794561779_14300\pot_plant_1_0.urdf
INFO:root:Category sofa
INFO:root:Loading the following URDF template C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\objects\sofa\427c7655012b6cc5593ebeeedbff73b\427c7655012b6cc5593ebeeedbff73b.urdf
INFO:root:Scale: [2.13580209 2.46916803 2.29157828]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scene_instances\20240828-135717_12121768275794561779_14300\sofa_2_0.urdf
INFO:root:Category floor_lamp
INFO:root:Loading the following URDF template C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\objects\floor_lamp\c7aadb7ac29ac778aaae3d020f5ddf8\c7aadb7ac29ac778aaae3d020f5ddf8.urdf
INFO:root:Scale: [1.05209914 1.2544259  1.79462192]

Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\shape/visual\../../material/wall/COMBINED.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\shape/visual\../../material/wall/NORMAL.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\shape/visual\../../material/floor_2/COMBINED.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\shape/visual\../../material/floor_2/NORMAL.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\shape/visual\../../material/floor_1/COMBINED.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\shape/visual\../../material/floor_1/NORMAL.png
Texture:  C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data/ig_dataset\scenes\Rs_int\shape/visual\../../material/floor_0/COMBINED.png
Texture:  C:\Users\Bryan\Documen

In [60]:
class DynamicListStorage:
    def __init__(self, initial_capacity: int = 1000, list_length: int = 960):
        self.list_length = list_length
        self.capacity = initial_capacity
        self.data = np.zeros((self.capacity, self.list_length), dtype=np.int8)
        self.current_index = 0

    def add_list(self, lst):
        if self.current_index >= self.capacity:
            self._expand_capacity()
        
        self.data[self.current_index] = lst
        self.current_index += 1

    def _expand_capacity(self):
        new_capacity = self.capacity * 2
        new_data = np.zeros((new_capacity, self.list_length), dtype=np.int8)
        new_data[:self.capacity] = self.data
        self.data = new_data
        self.capacity = new_capacity

    def get_list(self, index: int) -> np.ndarray:
        if index >= self.current_index:
            raise IndexError("List index out of range")
        return self.data[index]

    def __len__(self):
        return self.current_index

def cosine_similarity(col1: np.ndarray, col2: np.ndarray) -> float:
    dot_product = np.dot(col1, col2)
    magnitude1 = np.linalg.norm(col1)
    magnitude2 = np.linalg.norm(col2)
    
    if magnitude1 * magnitude2 == 0:
        return 0
    else:
        return dot_product / (magnitude1 * magnitude2)
    
def detect_key_frames(storage: DynamicListStorage, threshold: float = 0.95):
    key_frames = []
    reference_list = storage.get_list(0)
    
    for i in range(1, len(storage)):
        current_list = storage.get_list(i)
        similarity = cosine_similarity(reference_list, current_list)
        
        if similarity < threshold:
            key_frames.append(i)
            reference_list = current_list
    
    return key_frames

In [63]:
storage.current_index

1

In [14]:
for _, v in igbhvr_act_inst.object_scope.items():
    print("\n" ,v.name,":  ")
    for state in v.states:
        print(state)
        print(inspect_class(state))
    
    raise


 oatmeal_0 :  
<class 'igibson.object_states.temperature.Temperature'>
- _abc_impl: _abc_data
- _dump: function
  Signature: (self)
- _get_value: function
  Signature: (self)
- _initialize: function
  Signature: (self)
- _set_value: function
  Signature: (self, new_value)
- _update: function
  Signature: (self)
- dump: function
  Signature: (self)
- get_dependencies: function
  Signature: ()
- get_optional_dependencies: function
  Signature: ()
- get_value: function
  Signature: (self, *args, **kwargs)
- initialize: function
  Signature: (self, simulator)
- load: function
  Signature: (self, data)
- set_value: function
  Signature: (self, *args, **kwargs)
- update: function
  Signature: (self)
None
<class 'igibson.object_states.max_temperature.MaxTemperature'>
- _abc_impl: _abc_data
- _dump: function
  Signature: (self)
- _get_value: function
  Signature: (self)
- _initialize: function
  Signature: (self)
- _set_value: function
  Signature: (self, new_value)
- _update: function
  Sign

RuntimeError: No active exception to reraise

In [ ]:
print(igbhvr_act_inst.object_scope.keys())
print(igbhvr_act_inst.object_scope.values())
print(igbhvr_act_inst.object_scope.items())

dict_keys(['oatmeal.n.01_1', 'oatmeal.n.01_2', 'countertop.n.01_1', 'chip.n.04_1', 'chip.n.04_2', 'vegetable_oil.n.01_1', 'vegetable_oil.n.01_2', 'sugar.n.01_1', 'sugar.n.01_2', 'cabinet.n.01_1', 'floor.n.01_1', 'agent.n.01_1'])
dict_values([<igibson.objects.articulated_object.URDFObject object at 0x0000025981C25F70>, <igibson.objects.articulated_object.URDFObject object at 0x0000025985380280>, <igibson.objects.articulated_object.URDFObject object at 0x0000025986234C10>, <igibson.objects.articulated_object.URDFObject object at 0x00000259853801C0>, <igibson.objects.articulated_object.URDFObject object at 0x0000025985EBC490>, <igibson.objects.articulated_object.URDFObject object at 0x00000259857749A0>, <igibson.objects.articulated_object.URDFObject object at 0x0000025985774130>, <igibson.objects.articulated_object.URDFObject object at 0x000002598269DBE0>, <igibson.objects.articulated_object.URDFObject object at 0x0000025982685EB0>, <igibson.objects.articulated_object.URDFObject object at

In [27]:
for _, v in igbhvr_act_inst.object_scope.items():
    print("\n" ,v.name,":  \n")
    # print(v.states.values())
    for key, value in v.states.items():
        print(key)
        try:
            print("    ", value.get_value())
        except:
            for _, w in igbhvr_act_inst.object_scope.items():
                print("  ", w.name,":  ",value.get_value(w))
    


 oatmeal_0 :  

<class 'igibson.object_states.temperature.Temperature'>
     23.0
<class 'igibson.object_states.max_temperature.MaxTemperature'>
     23.0
<class 'igibson.object_states.room_states.InsideRoomTypes'>
     [None]
<class 'igibson.object_states.adjacency.HorizontalAdjacency'>
     [(AxisAdjacencyList(positive_neighbors=[65, 68], negative_neighbors=[63, 66]), AxisAdjacencyList(positive_neighbors=[67], negative_neighbors=[])), (AxisAdjacencyList(positive_neighbors=[64], negative_neighbors=[]), AxisAdjacencyList(positive_neighbors=[69], negative_neighbors=[])), (AxisAdjacencyList(positive_neighbors=[], negative_neighbors=[]), AxisAdjacencyList(positive_neighbors=[69], negative_neighbors=[])), (AxisAdjacencyList(positive_neighbors=[], negative_neighbors=[]), AxisAdjacencyList(positive_neighbors=[], negative_neighbors=[])), (AxisAdjacencyList(positive_neighbors=[67], negative_neighbors=[]), AxisAdjacencyList(positive_neighbors=[63, 66], negative_neighbors=[65, 68]))]
<class 'ig

In [38]:
data = []

for _, v in igbhvr_act_inst.object_scope.items():
    print("\n" ,v.name,":  \n")
    # print(v.states.values())
    for key, value in v.states.items():
        try:
            if value.get_value() in [True, False]:
                print(key, "\n    ", value.get_value())
                data.append(((v.name, key), value.get_value()))
        except:
            if value.get_value(v) in [True, False]:
                print(key)
                for _, w in igbhvr_act_inst.object_scope.items():
                    print("    ", w.name,":  ",value.get_value(w))
                    data.append(((v.name, key, w.name), value.get_value(w)))


 oatmeal_0 :  

<class 'igibson.object_states.frozen.Frozen'> 
     False
<class 'igibson.object_states.cooked.Cooked'> 
     False
<class 'igibson.object_states.burnt.Burnt'> 
     False
<class 'igibson.object_states.robot_related_states.InHandOfRobot'> 
     False
<class 'igibson.object_states.on_floor.OnFloor'>
     oatmeal_0 :   False
     oatmeal_1 :   False
     countertop_26 :   False
     chip_0 :   False
     chip_1 :   False
     olive_oil_0 :   False
     olive_oil_1 :   False
     sugar_jar_0 :   False
     sugar_jar_1 :   False
     bottom_cabinet_41 :   False
     room_floor_kitchen_0 :   False
     BRBody_1 :   False
<class 'igibson.object_states.robot_related_states.InReachOfRobot'> 
     True
<class 'igibson.object_states.robot_related_states.InSameRoomAsRobot'> 
     False
<class 'igibson.object_states.robot_related_states.InFOVOfRobot'> 
     False
<class 'igibson.object_states.under.Under'>
     oatmeal_0 :   False
     oatmeal_1 :   False
     countertop_26 :   Fa

In [39]:
data

[(('oatmeal_0', igibson.object_states.frozen.Frozen), False),
 (('oatmeal_0', igibson.object_states.cooked.Cooked), False),
 (('oatmeal_0', igibson.object_states.burnt.Burnt), False),
 (('oatmeal_0', igibson.object_states.robot_related_states.InHandOfRobot),
  False),
 (('oatmeal_0', igibson.object_states.on_floor.OnFloor, 'oatmeal_0'), False),
 (('oatmeal_0', igibson.object_states.on_floor.OnFloor, 'oatmeal_1'), False),
 (('oatmeal_0', igibson.object_states.on_floor.OnFloor, 'countertop_26'),
  False),
 (('oatmeal_0', igibson.object_states.on_floor.OnFloor, 'chip_0'), False),
 (('oatmeal_0', igibson.object_states.on_floor.OnFloor, 'chip_1'), False),
 (('oatmeal_0', igibson.object_states.on_floor.OnFloor, 'olive_oil_0'), False),
 (('oatmeal_0', igibson.object_states.on_floor.OnFloor, 'olive_oil_1'), False),
 (('oatmeal_0', igibson.object_states.on_floor.OnFloor, 'sugar_jar_0'), False),
 (('oatmeal_0', igibson.object_states.on_floor.OnFloor, 'sugar_jar_1'), False),
 (('oatmeal_0', igibs

In [41]:
bool_column = [int(row[1]) for row in data]
bool_column

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [48]:
def get_flattened_scene_graph(igbhvr_act_inst):
    data = []

    for _, v in igbhvr_act_inst.object_scope.items():
        for key, value in v.states.items():
            try:
                if value.get_value() in [True, False]:
                    data.append(((v.name, key), value.get_value()))
            except:
                if value.get_value(v) in [True, False]:
                    for _, w in igbhvr_act_inst.object_scope.items():
                        data.append(((v.name, key, w.name), value.get_value(w)))
    
    return [int(row[1]) for row in data]


col = get_flattened_scene_graph(igbhvr_act_inst)
col
len(col)

960

In [ ]:
import math

def cosine_similarity(vec1, vec2):
    
    # Compute dot product
    dot_product = sum(a * b for a, b in zip(vec1, vec2))
    
    # Compute magnitudes
    magnitude1 = math.sqrt(sum(a * a for a in vec1))
    magnitude2 = math.sqrt(sum(b * b for b in vec2))
    
    # Compute cosine similarity
    if magnitude1 * magnitude2 == 0:
        return 0  # Handle the case where one or both vectors are all zeros
    else:
        return dot_product / (magnitude1 * magnitude2)




In [ ]:
for _, v in igbhvr_act_inst.object_scope.items():
    print("\n" ,v.name,":  ")
    for state in v.states:
        print(state)


 oatmeal_0 :  
<class 'igibson.object_states.temperature.Temperature'>
<class 'igibson.object_states.max_temperature.MaxTemperature'>
<class 'igibson.object_states.room_states.InsideRoomTypes'>
<class 'igibson.object_states.adjacency.VerticalAdjacency'>
<class 'igibson.object_states.adjacency.HorizontalAdjacency'>
<class 'igibson.object_states.contact_bodies.ContactBodies'>
<class 'igibson.object_states.aabb.AABB'>
<class 'igibson.object_states.pose.Pose'>
<class 'igibson.object_states.frozen.Frozen'>
<class 'igibson.object_states.cooked.Cooked'>
<class 'igibson.object_states.burnt.Burnt'>
<class 'igibson.object_states.robot_related_states.InHandOfRobot'>
<class 'igibson.object_states.touching.Touching'>
<class 'igibson.object_states.on_top.OnTop'>
<class 'igibson.object_states.on_floor.OnFloor'>
<class 'igibson.object_states.under.Under'>
<class 'igibson.object_states.robot_related_states.InFOVOfRobot'>
<class 'igibson.object_states.robot_related_states.InReachOfRobot'>
<class 'igibs

In [ ]:
inspect_class(igbhvr_act_inst, indent=4)

        - activity_definition: int32
        - behavior_activity: str
        - check_scene: method
          Signature: ()
        - check_success: method
          Signature: ()
        - clutter_scene: method
          Signature: ()
        - current_goal_status: dict
        - current_success: bool
        - currently_viewed_index: int
        - currently_viewed_instruction: int32
        - debug_sampling: method
          Signature: (scene_object_scope_filtered, non_sampleable_obj_conditions, goal_condition_set=None)
        - gen_goal_conditions: method
          Signature: ()
        - gen_ground_goal_conditions: method
          Signature: ()
        - gen_initial_conditions: method
          Signature: ()
        - get_task_obs: method
          Signature: (env)
        - goal_conditions: list
        - ground_goal_state_options: list
        - import_agent: method
          Signature: ()
        - import_scene: method
          Signature: ()
        - initial_conditions: list

In [ ]:
inspect_class(igbhvr_act_inst.scene, indent=4)

        - _abc_impl: _abc_data
        - _add_object: method
          Signature: (obj)
        - _load: method
          Signature: ()
        - _set_first_n_objects: method
          Signature: (first_n_objects)
        - _set_obj_names_to_load: method
          Signature: (obj_name_list)
        - add_object: method
          Signature: (obj, _is_call_from_simulator=False)
        - agent: dict
        - avg_obj_dims: dict
        - body_collision_set: set
        - body_id_to_name: dict
        - build_graph: bool
        - build_trav_graph: method
          Signature: (maps_path, floor, trav_map)
        - category_ids: defaultdict
        - check_collision: method
          Signature: (body_a, body_b=None, link_a=None, fixed_body_ids=None)
        - check_scene_quality: method
          Signature: (body_ids, fixed_body_ids)
        - filter_rooms_and_object_categories: method
          Signature: (load_object_categories, not_load_object_categories, load_room_types, load_room_inst

In [ ]:
def import_scene(self):
    self.simulator.reload()
    self.simulator.import_ig_scene(self.scene)

    if not self.online_sampling:
        for obj_inst in self.object_scope:
            matched_sim_obj = None
            
            if "floor.n.01" in obj_inst:
                for _, sim_obj in self.scene.objects_by_name.items():
                    if sim_obj.bddl_object_scope is not None and obj_inst in sim_obj.bddl_object_scope:
                        bddl_object_scope = sim_obj.bddl_object_scope.split(",")
                        bddl_object_scope = {item.split(":")[0]: item.split(":")[1] for item in bddl_object_scope}
                        assert obj_inst in bddl_object_scope
                        room_inst = bddl_object_scope[obj_inst].replace("room_floor_", "")
                        matched_sim_obj = RoomFloor(
                            category="room_floor",
                            name=bddl_object_scope[obj_inst],
                            scene=self.scene,
                            room_instance=room_inst,
                            floor_obj=self.scene.objects_by_name["floors"],
                        )
            elif obj_inst == "agent.n.01_1":
                # Skip adding agent to object scope, handled later by import_agent()
                continue
            else:
                for _, sim_obj in self.scene.objects_by_name.items():
                    if sim_obj.bddl_object_scope == obj_inst:
                        matched_sim_obj = sim_obj
                        break
            assert matched_sim_obj is not None, obj_inst
            self.object_scope[obj_inst] = matched_sim_obj

# Actual Code

In [ ]:
# Setup

import argparse
import datetime
import os
import pprint

import bddl
import h5py
import numpy as np

import igibson
from igibson.activity.activity_base import iGBEHAVIORActivityInstance
from igibson.render.mesh_renderer.mesh_renderer_cpu import MeshRendererSettings
from igibson.render.mesh_renderer.mesh_renderer_vr import VrSettings
from igibson.simulator import Simulator
from igibson.utils.git_utils import project_git_info
from igibson.utils.ig_logging import IGLogReader, IGLogWriter
from igibson.utils.utils import parse_str_config


def print_class_info(obj, indent=0):
    # Get the class of the object
    cls = obj.__class__

    # Print the class name
    print(" " * indent + f"Class: {cls.__name__}")

    # Iterate through the object's attributes
    for attr_name, attr_value in vars(obj).items():
        print(" " * (indent + 2) + f"{attr_name}:")
        
        # If the attribute is an instance of a custom class, recurse
        if hasattr(attr_value, '__dict__'):
            print_class_info(attr_value, indent + 4)
        else:
            print(" " * (indent + 4) + f"{attr_value}")

    # Print information about methods
    methods = [method for method in dir(cls) if callable(getattr(cls, method)) and not method.startswith("__")]
    if methods:
        print(" " * (indent + 2) + "Methods:")
        for method in methods:
            print(" " * (indent + 4) + method)


def verify_determinism(in_log_path, out_log_path):
    is_deterministic = True
    with h5py.File(in_log_path) as original_file, h5py.File(out_log_path) as new_file:
        for obj in original_file["physics_data"]:
            for attribute in original_file["physics_data"][obj]:
                is_close = np.isclose(
                    original_file["physics_data"][obj][attribute], new_file["physics_data"][obj][attribute]
                )
                is_deterministic = is_deterministic and is_close.all()
                if not is_close.all():
                    print(
                        "Mismatch for obj {} with mismatched attribute {} starting at timestep {}".format(
                            obj, attribute, np.where(is_close == False)[0][0]
                        )
                    )
    return bool(is_deterministic)


def safe_replay_demo(*args, **kwargs):
    """Replays a demo, asserting that it was deterministic."""
    demo_statistics = replay_demo(*args, **kwargs)
    assert (
        demo_statistics["deterministic"] == True
    ), "Replay was not deterministic (or was executed with disable_save=True)."


# Define a class to mock argparse's Namespace object
class Args:
    def __init__(self, vr_log_path, vr_replay_log_path=None, frame_save_path=None, disable_save=False, profile=False, mode=None):
        self.vr_log_path = vr_log_path
        self.vr_replay_log_path = vr_replay_log_path
        self.frame_save_path = frame_save_path
        self.disable_save = disable_save
        self.profile = profile
        self.mode = mode

# Manually specify the arguments here
args = Args(
    vr_log_path=r"C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data\ig_dataset\tests\storing_food_0_Rs_int_2021-05-31_11-49-30.hdf5",
    frame_save_path=r"C:\Users\Bryan\Documents\Code\demo\saved_frames\storing_food",
    mode="simple"
)



def main():
    args = Args(vr_log_path=r"C:\Users\Bryan\Documents\Code\demo\iGibson\igibson\data\ig_dataset\tests\storing_food_0_Rs_int_2021-05-31_11-49-30.hdf5", frame_save_path=r"C:\Users\Bryan\Documents\Code\demo\saved_frames\storing_food", mode="simple"
)
    bddl.set_backend("iGibson")
    igbhvr_act_inst = replay_demo(
        args.vr_log_path,
        out_log_path=args.vr_replay_log_path,
        disable_save=args.disable_save,
        frame_save_path=args.frame_save_path,
        mode=args.mode,
        profile=args.profile,
    )
    
    return igbhvr_act_inst